In [1]:
import os
import configparser
import pandas
from pyspark.sql import SparkSession

In [2]:
config = configparser.ConfigParser()
config.read('../../dl.cfg')

os.environ['AWS_ACCESS_KEY_ID'] = config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = config['AWS']['AWS_SECRET_ACCESS_KEY']
    
input_data = config['LOCAL']['INPUT_DATA']
output_data = config['LOCAL']['OUTPUT_DATA']

In [3]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

:: loading settings :: url = jar:file:/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/neeliyer/.ivy2/cache
The jars for the packages stored in: /Users/neeliyer/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ac87ed5f-a9e9-419b-a173-df17613a8fe6;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;2.7.0 in central
	found org.apache.hadoop#hadoop-common;2.7.0 in central
	found org.apache.hadoop#hadoop-annotations;2.7.0 in central
	found com.google.guava#guava;11.0.2 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found commons-cli#commons-cli;1.2 in central
	found org.apache.commons#commons-math3;3.1.1 in central
	found xmlenc#xmlenc;0.52 in central
	found commons-httpclient#commons-httpclient;3.1 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.4 in central
	found commons-io#commons-io;2.4 in central
	found commons-net#commons-net;3.1 in central
	found commons-collections#commons-c

In [9]:
clickstream_df = spark.read.parquet(os.path.join('../', output_data, 'clickstream/year=2018/month=4/*.parquet'))
print('size=', clickstream_df.count())
clickstream_df.limit(5).toPandas()

size= 89949


,anonymous_user_id,url,time,browser,os,screen_resolution,datetime,utm_source,utm_medium,path
0,146ee066-7d39-4785-a845-4a298d4d9f00,https://www.cultureamp.com/employee-feedback/?...,1523324215,Mobile Safari,iOS,375x667,2018-04-10,google,cpc,None
1,8f41c5e4-2412-437e-ba52-f1aa20338118,https://www.cultureamp.com/employee-feedback/?...,1523322979,Chrome Mobile,Android,320x569,2018-04-10,google,cpc,None
2,c528e170-9796-49c7-94fd-1fd18ceb7d97,https://www.cultureamp.com/employee-feedback/?...,1523322339,Mobile Safari,iOS,768x1024,2018-04-10,google,cpc,None
3,7ebaeeb8-5e1a-4009-9db0-616f7a12aeb8,https://www.cultureamp.com/thanks.html?submiss...,1523323617,Chrome Mobile,Android,412x732,2018-04-10,None,None,None
4,5f15657e-052d-4235-b2da-e22ed601872c,https://www.cultureamp.com/employee-feedback/?...,1523324762,Mobile Safari,iOS,375x667,2018-04-10,google,cpc,None


In [10]:
clickstream_df.registerTempTable("clickstream_df")
spark.sql("select * from clickstream_df limit 3").show()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


+--------------------+--------------------+----------+-------------+-------+-----------------+----------+----------+----------+----+
|   anonymous_user_id|                 url|      time|      browser|     os|screen_resolution|  datetime|utm_source|utm_medium|path|
+--------------------+--------------------+----------+-------------+-------+-----------------+----------+----------+----------+----+
|146ee066-7d39-478...|https://www.cultu...|1523324215|Mobile Safari|    iOS|          375x667|2018-04-10|    google|       cpc|null|
|8f41c5e4-2412-437...|https://www.cultu...|1523322979|Chrome Mobile|Android|          320x569|2018-04-10|    google|       cpc|null|
|c528e170-9796-49c...|https://www.cultu...|1523322339|Mobile Safari|    iOS|         768x1024|2018-04-10|    google|       cpc|null|
+--------------------+--------------------+----------+-------------+-------+-----------------+----------+----------+----------+----+



In [11]:
def sql_helper(sql, limit = 10):
    if None: return spark.sql(sql).show()
    else: return spark.sql(sql).show(limit)

In [12]:
sql = "select count(distinct anonymous_user_id) from clickstream_df"
sql_helper(sql)

+---------------------------------+
|count(DISTINCT anonymous_user_id)|
+---------------------------------+
|                            55367|
+---------------------------------+



In [20]:
sql = \
"""
select count(anonymous_user_id), utm_source
from clickstream_df
where utm_source != 'null'
group by utm_source
"""
sql_helper(sql)

+------------------------+-------------+
|count(anonymous_user_id)|   utm_source|
+------------------------+-------------+
|                      62|hs_automation|
|                    1001|     hs_email|
|                      18|      twitter|
|                       2|          ppc|
|                     363|     linkedin|
|                      17|     capterra|
|                       6|  twitter.com|
|                    3897|     homepage|
|                       4|   Office Kit|
|                      93|     facebook|
+------------------------+-------------+
only showing top 10 rows

